### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

### Compile contract from file

In [255]:
import json
import web3
from datetime import datetime
from web3 import Web3
from solcx import compile_source
import solcx
#from web3.contract import ConciseContract

In [256]:
# Read in Model contract code
with open('./data/Model.sol', 'r') as file:
    contract_source_code = file.read()

In [257]:
solcx.install_solc('0.6.2')
# Compile & Store Compiled source code
compiled_sol = solcx.compile_source(contract_source_code,solc_version='0.6.2')

In [258]:
#solcx.install_solc('0.6.2')

In [259]:
#compiled_sol

In [260]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Model']

In [261]:
#contract_interface

In [262]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [263]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))

In [264]:
# Use local Ganache GUI on macOS
#w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

In [265]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [266]:
w3.eth.accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

In [267]:
w3.eth.accounts[0]

'0x23cA5505ca6D97f439C66AaA6b371216406CDE4F'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [268]:
# Create contract blueprint
Contract = w3.eth.contract(abi=abi, bytecode=bytecode)

In [269]:
# Submit the transaction that deploys the contract
tx_hash = Contract.constructor().transact({"from": w3.eth.accounts[0]})

In [270]:
tx_hash

HexBytes('0x69bd55294cd2b7edc6aae678a7d9c08a7cbda0fb2aa0ae7166398cb902559aaf')

### Obtain Transcation Receipt

In [271]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [272]:
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number",contract_block,".")

The contract is deployed with block number 47 .


In [273]:
# With obtain the final address of the contract 
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

The contract has the address 0x5EDc4382c85010772428686ceCA7c2747B883013 .


### Interact with contract

In [274]:
# Create python instance of deployed contract
Contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [275]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [276]:
accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

**Certification**  
```solidity

        function addCertifier(address newCertifier) public onlyOwner {}
        function deleteCertifier(address oldCertifier) public onlyOwner {}
        function checkIfCertified(address certified) public view returns(bool) {}
```

In [277]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
startTime = datetime.now()

In [278]:
#checking if adding a new account to the certification works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before adding, certified = " + str(tx_hash))
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after adding, certified = " + str(tx_hash))

before adding, certified = False
after adding, certified = True


In [279]:
#checking if removing a certifier also works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before removing, certified = " + str(tx_hash))
tx_hash = Contract.functions.deleteCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after removing, certified = " + str(tx_hash))

before removing, certified = True
after removing, certified = False


In [280]:
print(w3.eth.get_block('latest'))

AttributeDict({'hash': HexBytes('0x79dce14f203f8acaddc845126b15d409c7a6c8545551620c5719c5773bb95f81'), 'parentHash': HexBytes('0xefe6c9c993b08ff02667d07b4d99ccc7e4e26f731db39d861aacfefeef182966'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'miner': '0x0000000000000000000000000000000000000000', 'stateRoot': HexBytes('0xe07eef84210278565dd81fb4aa47809379009c72e1a4bb7709b1feebfe216c2d'), 'transactionsRoot': HexBytes('0x7af7630d21850b6daf6d09edeb9c585b0a4d95f51150aa69b74d0c0fe16a5928'), 'receiptsRoot': HexBytes('0xf4b08aeccec4c3ac186956b0303ad0f848a910c60f4c057efdd4efe7bc467e96'), 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [281]:
endTimeDataProvider = datetime.now()
elapsedTimeDataProvider = endTimeDataProvider - startTime
print("elapsedTimeDataProvider", elapsedTimeDataProvider)

elapsedTimeDataProvider 0:00:00.636479


In [282]:
#tx_hash

**Adding models**  
```solidity
        function publishModel(
                string memory _link,
                bytes memory _hash) public onlyOwner providerGasCost
```

**Updating Models**  
```solidity
        function updateHash(
                string memory _link,
                bytes memory newHash) public onlyOwner providerGasCost
```

**Certifying models**  
```solidity
        function certifyModel(bytes memory _hash) public certifier
```


**Getters and Setters for variables**  
```solidity
        function getVersion(bytes32 _hash)public view returns(uint256){}
        function getCertfication(uint256 _version) public view returns(bool){}
        function setLink(string memory _link) public onlyOwner(){}
        function getLink() public view returns(string memory){}
        function getModelCardHash() public view returns(bytes32){}


In [283]:
#Lets look at how the functions work whenever we are trying to upload a model,
#In here im using a test hash, this does not refer to a real hash, //you can add a real model here 

test_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777b93592d8d4d4a4096c57876a91c84f4a712')
#basically if model card hash is not 32 bytes needs to be changed
test_link = 'https/:www.somelinktomodel.com'
#please let me know if you have any troubles with this part, it might be that the model card is not a bytes32 hash, in that case i need to change the smart contract,
tx_hash = Contract.functions.publishModel(test_link, test_hash).transact({'from': accounts[0]})
#this model is not callable twice on the same contract, as it will recognise that the contract is published, keep that in mind while testing other functions in the same cell

#lets see if everything prints accordingly
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

https/:www.somelinktomodel.com, the link to the model
the model is certified: False


In [284]:
#let's see if we can certify the model, let's certify an account
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from': accounts[0]})
tx_hash = Contract.functions.certifyModel(test_hash).transact({'from': accounts[1]})
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

the model is certified: True


In [302]:
#Models can get certified, lets update a model
new_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777b93592d8d4d4sfdssc57876a91c84f4a712')
new_link = 'https/:www.somelinktonewmodel.com'
tx_hash = Contract.functions.updateHash(new_link, new_hash).transact({'from': accounts[0]})
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the " + str(m_version) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version).call()))
m_version_new = Contract.functions.getVersion(new_hash).call()
print("the " + str(m_version_new) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version_new).call()))
#making sure that the hash from the contract is the same as the input
assert str(Contract.functions.getMostRecentModelCardHash().call()) == str(new_hash)


https/:www.somelinktonewmodel.com, the link to the model
the 1th version of model is certified: True
the 15th version of model is certified: False


In [251]:
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)

elapsedTime1 0:12:59.385628


In [ ]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address8, False, False, False, False, True).estimateGas())
#print("Gas", Contract.functions.UploadDataRequirements(test_address0, False, True, False, False, False, False, False, False).estimateGas())
#print("Gas", Contract.functions.giveResearchPurpose(test_address13, True, True, False, False, False).estimateGas())
#print("Gas", Contract.functions.givePerson(test_address13, True, False, True, False).estimateGas())
#print("Gas", Contract.functions.giveProfit(test_address13, True, False).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address13).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address14).estimateGas())
#print("Gas", Contract.functions.AccessData(test_address1, test_address15).estimateGas())